In [ ]:
#hide
!pip install fastai==2.2.5

In [ ]:
from fastai.vision.all import *

In [ ]:
from ipywidgets import FileUpload, Output, Label, VBox, Button

In [ ]:
path = untar_data(URLs.PASCAL_2007)

In [ ]:
path.ls()


In [ ]:
(path / "train") .ls()

In [ ]:
Image.open((path / "train/009463.jpg"))

In [ ]:
df = pd.read_csv(path / "train.csv")

In [ ]:
df.head(5)

In [ ]:
def get_x(r): return (path/"train") / r["fname"]
def get_y(r): return r["labels"].split(" ")
def splitter(df):
  train = df.index[~df["is_valid"]].tolist()
  valid = df.index[df["is_valid"]].tolist()
  return train,valid
  
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock), 
                         get_x=get_x, 
                         get_y=get_y, 
                         splitter=splitter,
                         item_tfms= RandomResizedCrop(128, min_scale=0.35))

In [ ]:
dls = dblock.dataloaders(df)

In [ ]:
dls.one_batch()[0].shape

In [ ]:
dls.show_batch()

## Baseline

In [ ]:
class AllZeroModel(nn.Module):
    def __init__(self):
        super(AllZeroModel, self).__init__()

    def forward(self, x):
        output = torch.zeros(x.shape[0], 20).to("cuda:0")
        return output

In [ ]:
model = AllZeroModel()

In [ ]:
learn = Learner(dls, AllZeroModel(), metrics=[accuracy_multi])

In [ ]:
learn.show_results()

In [ ]:
learn.validate()

## CNN Learner

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=[accuracy_multi])
learn.fine_tune(3, freeze_epochs=2)

In [ ]:
# BCEWithLogitsLoss and MultiAccuracy before training
learn.validate()

In [ ]:
learn.show_results()

In [ ]:
upload = FileUpload()
out_pl = Output()
lbl_pred = Label()
btn_run = Button(description="classify")

def on_click_classify(change):
  img = PILImage.create(upload.data[-1])
  out_pl.clear_output()
  with out_pl: display(img.to_thumb(128, 128))
  pred, pred_idx, probs = learn.predict(tensor(img))
  lbl_pred.value = f'Prediction {pred}, Probability: {probs[pred_idx]}'

btn_run.on_click(on_click_classify)

In [ ]:
VBox([upload, btn_run, out_pl, lbl_pred])

In [ ]:
learn.predict(upload.data[-1])
print(pred)
print(list(zip(dls.vocab, probs)))